In [21]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re
import time
import random

# Scrapping the main page

In [3]:
samsung_support = pd.DataFrame({'title' : [], 'nb_pages': [], 'date': [], 'user':[], 'nb_views':[], 'nb_replies':[], 'link':[], 'thread':[]})

for i in range(1,134): # 134 pour max
    html = requests.get('https://us.community.samsung.com/t5/Galaxy-S8-Questions-and-Answers/bd-p/GS8QA/page/'+str(i)) # pour pouvoir itérer
    soup = BeautifulSoup(html.text, 'lxml')
    
    bs_object = soup.find_all('div', attrs = {"class": u"lia-component-messages-column-message-info"}) # Attention au u
    df = pd.DataFrame({'title' : [], 'date': [], 'user':[], 'nb_views':[], 'nb_replies':[], 'link':[]})

    for s,index in zip(bs_object,range(len(bs_object))): 
        df.loc[index,'title'] = s.find('div', attrs={"class":u"MessageSubjectCell"}).get_text().strip().split('  ')[0]
        
        try:
            df.loc[index, 'nb_pages'] = int(s.find('ul', attrs = {'class':u'lia-list-standard-inline'}).find_all('a', attrs ={'class':u'lia-link-navigation'})[-1].get_text().strip())
        except AttributeError:
            df.loc[index, 'nb_pages'] = 0
        except ValueError:
            df.loc[index, 'nb_pages'] = 0
            
        try:
            df.loc[index,'date'] = s.find('span', attrs={"class":u"DateTime"}).find('span', attrs={"class":u"local-date"}).get_text().strip()
        except AttributeError:
            df.loc[index,'date'] = s.find('span', attrs={"class":u"DateTime"}).find('span', attrs={"class":u"local-friendly-date"}).get_text().strip()
            
        df.loc[index,'user'] = s.find('span', attrs={"class":u"UserName"}).get_text().strip()
        
        df.loc[index,'nb_views'] = s.find_all('span', attrs={"class":u"lia-message-stats-count"})[0].get_text().strip()
    
        df.loc[index,'nb_replies'] = s.find_all('span', attrs={"class":u"lia-message-stats-count"})[1].get_text().strip()
        
        lien = str(s.find('div', attrs={"class":u"MessageSubject"}).find('a').get('href').strip())
        
        df.loc[index,'link'] = lien
        
        try:
            df.loc[index, 'thread'] = lien.split('/td-p/')[1]
        except IndexError:
            df.loc[index, 'thread'] = lien.split('/m-p/')[1]
        
    samsung_support = samsung_support.append(df, ignore_index = True)
    time.sleep(random.random()*3)

samsung_support = samsung_support.replace(r'\n',' ', regex=True)
samsung_support.shape

(3991, 8)

In [44]:
print(samsung_support.isnull().sum())
print(samsung_support.dtypes)
samsung_support.head()

date          0
link          0
nb_pages      0
nb_replies    0
nb_views      0
thread        0
title         0
user          0
dtype: int64
date           object
link           object
nb_pages      float64
nb_replies     object
nb_views       object
thread         object
title          object
user           object
dtype: object


,date,link,nb_pages,nb_replies,nb_views,thread,title,user
0,‎03-30-2017,/t5/Galaxy-S8-Questions-and-Answers/Galaxy-S8-...,8.0,8748,74,83749,Galaxy S8/S8+ Thoughts & Info,SamsungMarcos
1,yesterday,/t5/Galaxy-S8-Questions-and-Answers/Can-t-conn...,0.0,58,2,255011,Can't connect Android smartwatch with Android ...,userx2NMlglgnI
2,‎10-19-2017,/t5/Galaxy-S8-Questions-and-Answers/back-butto...,2.0,687,12,188372,back button open random app,usery1cMljASqS
3,Sunday,/t5/Galaxy-S8-Questions-and-Answers/LOVE-THE-P...,0.0,91,3,254448,LOVE THE PHONE BUT THE LAG IS OUTTA CONTROL AN...,user6tcgfrls7V
4,Sunday,/t5/Galaxy-S8-Questions-and-Answers/Apps-and-i...,0.0,33,1,254443,Apps and internet background,user9bpaDZldWq


In [6]:
samsung_support.to_csv("samsung_support.csv")

# Scrapping of the comments

In [35]:
data = pd.read_table('samsung_support.csv', sep = ',', encoding = 'WINDOWS-1252')
data.head()

,Unnamed: 0,date,link,nb_pages,nb_replies,nb_views,thread,title,user
0,0,?03-30-2017,/t5/Galaxy-S8-Questions-and-Answers/Galaxy-S8-...,8.0,8748,74,83749,Galaxy S8/S8+ Thoughts & Info,SamsungMarcos
1,1,yesterday,/t5/Galaxy-S8-Questions-and-Answers/Can-t-conn...,0.0,58,2,255011,Can't connect Android smartwatch with Android ...,userx2NMlglgnI
2,2,?10-19-2017,/t5/Galaxy-S8-Questions-and-Answers/back-butto...,2.0,687,12,188372,back button open random app,usery1cMljASqS
3,3,Sunday,/t5/Galaxy-S8-Questions-and-Answers/LOVE-THE-P...,0.0,91,3,254448,LOVE THE PHONE BUT THE LAG IS OUTTA CONTROL AN...,user6tcgfrls7V
4,4,Sunday,/t5/Galaxy-S8-Questions-and-Answers/Apps-and-i...,0.0,33,1,254443,Apps and internet background,user9bpaDZldWq


In [55]:
samsung_discussion = pd.DataFrame({'comment':[],  'link': []})

# for i in range(len(data.index)):
for i in range(1,1001):
    if int(data.nb_pages[i]) == 0:
        lien = str('https://us.community.samsung.com'+str(data.link[i]))
        html = requests.get(lien) # pour pouvoir itérer
        soup = BeautifulSoup(html.text, 'lxml')
        bs = soup.find_all('div', attrs = {'class':u'lia-message-body-content'})
        df = pd.DataFrame({'comment' : [], 'link':[]})
        
        for s, index in zip(bs, range(len(bs))):
            texte = s.find_all('p')
            comment = str()
            for k in range(len(texte)):
                comment += str(texte[k].get_text().strip())

            df.loc[index, 'comment'] = comment
            df.loc[index, 'link'] = lien
            
        samsung_discussion = samsung_discussion.append(df, ignore_index = True)
 
    else:
        for j in range(1,int(data.nb_pages[i])+1): 
            lien = str('https://us.community.samsung.com'+str(data.link[i])+'/page/'+str(j))  
            html = requests.get(lien) # pour pouvoir itérer
            soup = BeautifulSoup(html.text, 'lxml')
            bs = soup.find_all('div', attrs = {'class':u'lia-message-body-content'})
            df = pd.DataFrame({'comment' : [], 'link':[]})

            for s, index in zip(bs, range(len(bs))):
                texte = s.find_all('p')
                comment = str()
                for k in range(len(texte)):
                    comment += str(texte[k].get_text().strip())

                df.loc[index,'comment'] = comment
                df.loc[index, 'link'] = lien
            
            samsung_discussion = samsung_discussion.append(df, ignore_index = True)
    
    time.sleep(random.random()*3)

# div lia-message-body-content

print(samsung_discussion.shape)
samsung_discussion.head()

(1510, 2)


,comment,link
0,"Hey all, I've created a nice little thread her...",https://us.community.samsung.com/t5/Galaxy-S8-...
1,I can't wait for 4/20th but why the gold color...,https://us.community.samsung.com/t5/Galaxy-S8-...
2,Whats the security update schedule ?,https://us.community.samsung.com/t5/Galaxy-S8-...
3,Im excited also! I pre ordered on tmobile my s8,https://us.community.samsung.com/t5/Galaxy-S8-...
4,Can't wait! I preordered one as well. I also p...,https://us.community.samsung.com/t5/Galaxy-S8-...


In [ ]:
for i in range(2001,3001):
    if int(data.nb_pages[i]) == 0:
        lien = str('https://us.community.samsung.com'+str(data.link[i]))
        html = requests.get(lien) # pour pouvoir itérer
        soup = BeautifulSoup(html.text, 'lxml')
        bs = soup.find_all('div', attrs = {'class':u'lia-message-body-content'})
        df = pd.DataFrame({'comment' : [], 'link':[]})
        
        for s, index in zip(bs, range(len(bs))):
            texte = s.find_all('p')
            comment = str()
            for k in range(len(texte)):
                comment += str(texte[k].get_text().strip())

            df.loc[index, 'comment'] = comment
            df.loc[index, 'link'] = lien
            
        samsung_discussion = samsung_discussion.append(df, ignore_index = True)
 
    else:
        for j in range(1,int(data.nb_pages[i])+1): 
            lien = str('https://us.community.samsung.com'+str(data.link[i])+'/page/'+str(j))  
            html = requests.get(lien) # pour pouvoir itérer
            soup = BeautifulSoup(html.text, 'lxml')
            bs = soup.find_all('div', attrs = {'class':u'lia-message-body-content'})
            df = pd.DataFrame({'comment' : [], 'link':[]})

            for s, index in zip(bs, range(len(bs))):
                texte = s.find_all('p')
                comment = str()
                for k in range(len(texte)):
                    comment += str(texte[k].get_text().strip())

                df.loc[index,'comment'] = comment
                df.loc[index, 'link'] = lien
            
            samsung_discussion = samsung_discussion.append(df, ignore_index = True)
    
    time.sleep(random.random()*3)

# div lia-message-body-content

print(samsung_discussion.shape)
samsung_discussion.head()

In [ ]:
samsung_discussion = samsung_discussion.replace(r'\n',' ', regex=True)
samsung_discussion = samsung_discussion.replace('\xa0','', regex = True)
samsung_discussion.to_csv('samsung_discussion.csv')


In [58]:
samsung_discussion.comment[20]

#'\xa0' à supprimer

'I got notification that my S8+ will be delivered by end of day today. I am\xa0very much looking forward to being able to check everything out on my new device. I loved my S7 Edge, and dispite the update issue, I am still a loyal Samsung customer. This will make my 4th Galaxy S device. Will post thoughts and reviews after I get to play with it a bit.'